In [1]:
import edit.data

# Modifications

In addition to directly getting data with specifications of the variables, `modifications` can be used to modify the retrieval of data.

### The syntax is as follows:

Can be str of form

```
- '!accumulate[period: "6 hourly"]:tcwv>accum_tcwv'
```

Where the `!ABC` references the function to apply, the `[]` the init kwargs needed in json form,
and all after `:` being the normal variable specification with anything after `>` being the new name.

Or dictionary with following keys

```
    - source_var (REQUIRED)     Variable to modify
    - modification (REQUIRED)   Modification to apply
    - target_var                Rename of variable
    - **                        Any other keys for `modification`
```

This will be transparent to the user, and only act upon retrieval of data.

Available modifications include:

```
    - !accumulate
    - !mean
    - !aggregate
    - !constant
```

## Automatic Accumulations
When retrieving data from an index, a common operation is accumulating a variable over a temporal period. If the index is decorated with `edit.data.indexes.decorators.variable_modifications` this accumulation can be done in the `variable` specification with no other action needed. 




In [ ]:
ERA5_accumulations = edit.data.archive.NCI.ERA5('!accumulate[period: "6 hourly"]:tcwv>accum_tcwv')
ERA5_accumulations

ERA5
	Description                    ECWMF ReAnalysis v5
		 range                          1970-current
		 Documentation                  https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation


	Initialisation                 
		 level_value                    None
		 product                        reanalysis
		 transforms                     {}
		 variables                      !accumulate[period: "6 hourly"]:tcwv>accum_tcwv
	Transforms                     
		 ConformNaming                  {'latitude': ['lat', 'Latitude', 'yt_ocean', 'yt'], 'longitude': ['lon', 'Longitude', 'xt_ocean', 'xt'], 'time': ['Time']}
		 Rename                         {'t2m': '2t', 'u10': '10u', 'v10': '10v', 'siconc': 'ci'}
		 Modifier                       {'modifications': (Accumulate tcwv over 0 days 06:00:00,)}

In [ ]:
# Single timestep
ERA5_accumulations['2023-01-01T00']

<xarray.Dataset>
Dimensions:     (longitude: 1440, latitude: 721)
Coordinates:
  * longitude   (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
Data variables:
    accum_tcwv  (latitude, longitude) float32 dask.array<chunksize=(182, 360), meta=np.ndarray>

In [ ]:
# Full data of 6 hourly accumulations
ERA5_accumulations['2023-01-01']

<xarray.Dataset>
Dimensions:     (longitude: 1440, latitude: 721, time: 24)
Coordinates:
  * longitude   (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time        (time) datetime64[ns] 2023-01-01 ... 2023-01-01T23:00:00
Data variables:
    accum_tcwv  (time, latitude, longitude) float32 dask.array<chunksize=(5, 182, 360), meta=np.ndarray>
Attributes: (5)

## Constants

Some variables can be considered constants, and can be held fixed by the `edit.data.Indexes`, and held in memory to reduce compute time

In [2]:
ERA5_constants = edit.data.archive.NCI.ERA5('!constant[query: "2000-01-01T00"]:lsm')
ERA5_constants

ERA5
	Description                    ECWMF ReAnalysis v5
		 range                          '1970-current'
		 Documentation                  'https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation'


	Initialisation                 
		 level_value                    None
		 product                        'reanalysis'
		 variables                      '!constant[query: "2000-01-01T00"]:lsm'
	Transforms                     
		 StandardCoordinateNames        {'latitude': "['lat', 'Latitude', 'yt_ocean', 'yt']", 'longitude': "['lon', 'Longitude', 'xt_ocean', 'xt']", 'replacement_dictionary': 'None', 'time': "['Time']"}
		 Rename                         {'names': {'t2m': "'2t'", 'u10': "'10u'", 'v10': "'10v'", 'siconc': "'ci'"}}
		 Modifier                       {'modifications': "(Constant of 'lsm' from 2000-01-01T00,)"}

In [3]:
# Any other time will get the constant query time
ERA5_constants['2023-01-01']

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    time       datetime64[ns] 2023-01-01
Data variables:
    lsm        (latitude, longitude) float32 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0